In [3]:
import torch
import numpy as np

def get_device() -> torch.device:
    """
    Returns the appropriate torch device: CUDA if available, otherwise CPU.
    """
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
def embed_texts(
    texts: list[str],
    model: torch.nn.Module,
    tokenizer,
    batch_size: int = 16,
    max_length: int | None = None,
    device: torch.device | None = None
) -> np.ndarray:
    """
    Embed a list of texts into vector representations using the [CLS] token embedding.

    Args:
        texts (list[str]): Input texts to embed.
        model (torch.nn.Module): Pretrained Transformer model.
        tokenizer: Corresponding HuggingFace tokenizer.
        batch_size (int): Number of samples per batch.
        max_length (int, optional): Maximum token length. Defaults to model.config.max_position_embeddings.
        device (torch.device, optional): Device for inference. Defaults to CUDA if available else CPU.

    Returns:
        np.ndarray: Array of shape (len(texts), hidden_size) with embeddings.
    """
    # Determine max_length
    if max_length is None:
        max_length = model.config.max_position_embeddings

    # Determine device
    if device is None:
        device = get_device()

    # Prepare model
    model = model.to(device)
    model.eval()

    all_embeds: list[np.ndarray] = []
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i : i + batch_size]
            encoded = tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                max_length=max_length,
                return_tensors="pt"
            )
            # Move to device
            encoded = {k: v.to(device) for k, v in encoded.items()}

            outputs = model(**encoded)
            # Extract [CLS] token embedding (first token)
            cls_embeddings = outputs.last_hidden_state[:, 0, :]
            all_embeds.append(cls_embeddings.cpu().numpy())

    return np.vstack(all_embeds)

In [28]:
from transformers import AutoTokenizer, AutoModel

def load_model_and_tokenizer(model_name: str):
    """
    Load a pretrained model and its tokenizer from HuggingFace.

    Args:
        model_name (str): Name or path of the model.

    Returns:
        model, tokenizer
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    return model, tokenizer

In [7]:
import pandas as pd

df = pd.read_csv("labeled.csv")
df

,filename,label,text,basis
0,001_개인정보취급방침_가공.json,유리,제2조(개인정보의 처리 및 보유기간) \n① 협회는 법령에 따른 개인정보 보유․이용...,NaN
1,001_결혼정보서비스_가공.json,유리,제3조 (회원가입)\n① 회원이 되려고 하는 사람은 결혼관련 개인정보를 회사에 제공...,NaN
2,001_보증_가공.json,유리,제2조(보증금액)\n ① 이 보증서에 의한 보증금액은 채권자의 채무자에 대한 보증부...,NaN
3,001_사이버몰_가공.json,유리,제3조 (약관 등의 명시와 설명 및 개정)\n① 몰은 이 약관의 내용과 상호 및 대...,NaN
4,001_상해보험_가공.json,유리,제4조(보험금 지급에 관한 세부규정)\n② 제3조(보험금의 지급사유) 제2호에서 장...,NaN
...,...,...,...,...
7995,620_임대차계약_가공.json,불리,제2조(임대료)\n제2항 임대료 등의 연체시 매월 100분의 10에 해당하는 연체료...,매월 100분의 10은 연120퍼센트의 연체료율이 되어 이자제한법상 연25퍼센트를 ...
7996,621_임대차계약_가공.json,불리,제12조(해약인정 및 보증금 등의 반환)\n제2항 갑은 해약이 확정되면 즉시 을의 ...,점포를 재임대하여 반환하거나 또는 해약일로부터 3개월 이내에 임대보증금을 반환하되 ...
7997,622_임대차계약_가공.json,불리,제17조(임대인의 금지사항)\n제2항 을이 전항 및 제11조와 제14조에 위반 내지...,임대차계약관계에서 임차인이 지켜야 할 사항을 게을리하거나 지키지 않는 경우 임대인은...
7998,623_임대차계약_가공.json,불리,제20조(임대인의 금지조항)\n제1항 본 계약 각 조항의 해석상의 이의가 있는 경우...,해석상 이의가 있는 경우 임대인의 해석에 따르고 계약에 명시되지 않은 사항은 임대인...


In [10]:
from transformers import BertModel, BertTokenizer
from sklearn.neighbors import NearestNeighbors
import torch

In [11]:
model_name = "monologg/kobert"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [12]:
texts = df["text"].fillna("").astype(str).tolist()
embeddings = embed_texts(
    texts=texts,
    model=model,
    tokenizer=tokenizer,
    batch_size=16,
    max_length=model.config.max_position_embeddings
)

In [13]:
nn = NearestNeighbors(n_neighbors=5, metric="cosine")
nn.fit(embeddings)

NearestNeighbors(metric='cosine')

In [14]:
def find_similar(query: str, top_k: int = 5):
    # 1) 쿼리 임베딩
    query_emb = embed_texts(
        texts=[query],
        model=model,
        tokenizer=tokenizer,
        batch_size=1,
        max_length=model.config.max_position_embeddings
    )
    # 2) 최근접 이웃 검색
    distances, indices = nn.kneighbors(query_emb, n_neighbors=top_k)
    # 3) 거리 → 유사도로 변환 후 결과 조합
    results = []
    for dist, idx in zip(distances[0], indices[0]):
        sim = 1.0 - dist
        filename = df.loc[idx, "filename"]
        text     = df.loc[idx, "text"]
        results.append((filename, text, sim))
    return results


In [15]:
if __name__ == "__main__":
    query = "임대인은 임차인의 동의 없이 언제든지 본 계약상의 임대료 및 관리비 금액을 개정할 수 있다."
    find_similar(query, top_k=15)

In [16]:
import pandas as pd

# 1) 함수 호출
results = find_similar(query, top_k=15)

# 2) 리스트를 DataFrame으로 변환
df_results = pd.DataFrame(
    results,
    columns=["filename", "text", "similarity"]
)

# 3) 확인
df_results

,filename,text,similarity
0,056_가맹계약_가공.json,제10조 제5항\n물가의 상승 등 경제사정의 변동이 있는 경우 갑은 을에게 통고하고...,0.964538
1,108_신용카드_가공.json,제4조(카드의 유효기한 및 재발급)\n⑥ 카드가 갱신된 경우에도 계속하여 이 약관이...,0.954032
2,109_신용카드_가공.json,제4조(카드의 유효기한 및 재발급)\n⑥ 카드가 갱신된 경우에도 계속하여 이 약관을...,0.954032
3,384_임대차계약_가공.json,제4조(계약의 해제 및 해지)\n을이 다음 각호의 1에 해당하는 경우에는 갑은 이행...,0.953127
4,502_가맹계약_가공.json,제6조 (을사업자의 영업활동 조건에 관한 사항)\n③ 가맹점사업자의 의뢰가 있는 경...,0.953056
5,170_증권사2_가공.json,제20조(청약의 철회) \n① 고객은 금융소비자 보호에 관한 법률제46조 및 관련 ...,0.947912
6,172_증권사2_가공.json,제20조(청약의 철회) \n① 고객은 금융소비자 보호에 관한 법률제46조 및 관련 ...,0.947912
7,138_증권사2_가공.json,제12조(청약의 철회) \n① 저축자는 금융소비자 보호에 관한 법률제46조 및 관련...,0.947912
8,139_증권사2_가공.json,제12조(청약의 철회) \n① 저축자는 금융소비자 보호에 관한 법률제46조 및 관련...,0.947912
9,141_증권사2_가공.json,제12조(청약의 철회)\n① 저축자는 금융소비자 보호에 관한 법률제46조 및 관련 ...,0.947912


망했는데 -> 바이인코딩 방식이 문제일꺼야 -> 교차인코딩을 적용해보자

In [18]:
from sentence_transformers import CrossEncoder

# 1) CrossEncoder 모델 로드 (필요에 따라 다른 모델명 사용 가능)
cross_encoder = CrossEncoder(
    "cross-encoder/ms-marco-MiniLM-L-6-v2",
    device="cpu"
)


In [19]:
def rerank_with_cross_encoder(query: str,
                              top_k_bi: int = 15,
                              top_k_final: int = 5):
    """
    1) find_similar로 bi-encoder 기반 후보 top_k_bi개 추출
    2) CrossEncoder로 (query, candidate) 쌍에 점수 매김
    3) CrossEncoder 점수 내림차순 정렬 후 top_k_final개 반환
    """
    # 1) bi-encoder 기반 후보 추출
    #    find_similar은 (filename, text, bi_score) 튜플 리스트 반환
    candidates = find_similar(query, top_k=top_k_bi)
    
    # 2) CrossEncoder 입력 생성
    #    pairs: [(query, text1), (query, text2), ...]
    filenames, texts, bi_scores = zip(*candidates)
    pairs = [(query, t) for t in texts]
    
    # 3) CrossEncoder 예측
    cross_scores = cross_encoder.predict(pairs)  # 배열 길이 == top_k_bi
    
    # 4) bi_score, cross_score 함께 묶어 재정렬
    reranked = sorted(
        zip(filenames, texts, bi_scores, cross_scores),
        key=lambda x: x[3],  # cross_score 기준 내림차순
        reverse=True
    )
    
    # 5) 최종 top_k_final 반환
    return [
        {
            "filename":    fn,
            "text":        txt,
            "bi_score":    float(bs),
            "cross_score": float(cs)
        }
        for fn, txt, bs, cs in reranked[:top_k_final]
    ]


In [26]:
if __name__ == "__main__":
    query = "임대인은 임차인의 동의 없이 언제든지 본 계약상의 임대료 및 관리비 금액을 개정할 수 있다."
    results = rerank_with_cross_encoder(query, top_k_bi=15, top_k_final=5)
    
    # 리스트 → DataFrame 변환
    df_results = pd.DataFrame(results)
    
    # 소수점 네 자리까지 포맷팅
    pd.options.display.float_format = "{:.4f}".format
    

In [27]:
df_results

,filename,text,bi_score,cross_score
0,109_신용카드_가공.json,제4조(카드의 유효기한 및 재발급)\n⑥ 카드가 갱신된 경우에도 계속하여 이 약관을 적용할 수 있다.,0.9540,7.0063
1,384_임대차계약_가공.json,제4조(계약의 해제 및 해지)\n을이 다음 각호의 1에 해당하는 경우에는 갑은 이행의 최고 등 다른 절차를 취함이 없이 이 계약을 해제 또는 해지할 수 있다.,0.9531,6.9437
2,108_신용카드_가공.json,제4조(카드의 유효기한 및 재발급)\n⑥ 카드가 갱신된 경우에도 계속하여 이 약관이 적용될 수 있다.,0.9540,6.9235
3,502_가맹계약_가공.json,제6조 (을사업자의 영업활동 조건에 관한 사항)\n③ 가맹점사업자의 의뢰가 있는 경우에는 가맹본부가 직접 시공할 수 있다.\n,0.9531,6.7007
4,214_증권사2_가공.json,제14조(청약의 철회) \n① 가입자는 금융소비자 보호에 관한 법률 제46조 및 관련 규정이 정하는 바에 따라 청약철회가 가능한 수익증권에 한하여 계약서류를 제공받은 날(계약서류를 제공받지 아니한 경우에는 계약체결일)로부터 21일 이내 청약을 철회할 수 있다.,0.9470,6.6225


In [22]:
if __name__ == "__main__":
    query = "임대인 및 임차인은 본 계약에 관하여 발생하는 일체의 소송, 중재, 분쟁해결 절차와 관련하여 임대인의 본사 소재지를 관할하는 대한민국 법원을 단독·배타적 제1심 관할법원으로 하는 데 합의한다."
    results = rerank_with_cross_encoder(query, top_k_bi=15, top_k_final=5)
    
    # 리스트 → DataFrame 변환
    df_results = pd.DataFrame(results)
    
    # 소수점 네 자리까지 포맷팅
    pd.options.display.float_format = "{:.4f}".format

In [25]:
df_results

,filename,text,bi_score,cross_score
0,012_체육시설_가공.json,"제18조(사물함 대부 및 관리)\n6. 위임사항 월사용금 체불등의 사유로 임대차계약 해지시 임차인은 소유물품에 대한 이동, 보관의 권한을 임대인, 관리회사 및 중개업자에게 위임하며 15일 이내에 연체금 등을 정산하지 않을 시 그 소유권을 포기하고 일체의 민,형사상의 책임을 묻지 않기로 승낙한다.",0.9489,0.7515
1,091_게임_가공.json,"제16조 (저작권 등의 귀속)\n④ 게임 내에서 보여지지 않고 게임서비스와 일체화되지 않은 회원의 이용자 콘텐츠(예컨대, 일반게시판 등에서의 게시물)에 대하여 회사는 회원의 명시적인 동의가 없이 이용하지 않으며, 회원은 언제든지 이러한 이용자 콘텐츠를 삭제할 수 있습니다.",0.9495,0.0568
2,152_택배_가공.json,"제20조(손해배상)\n사업자는 자기 또는 사용인 기타 운송을 위하여 사용한 자가 운송물의 수탁, 인도, 보관 및 운송에 관하여 주의를 태만히 하지 않았음을 증명하지 못하는 한, 고객에게 운송물의 멸실, 훼손 또는 연착으로 인한 손해를 배상합니다.",0.9510,0.0047
3,288_게임_가공.json,"제15조 (서비스의 제공 및 중단 등) \n③ 회사는 제2항 제1호의 경우, 매주 또는 격주 단위로 일정 시간을 정하여 게임서비스를 중지할 수 있습니다. 이 경우 회사는 최소한 48시간 전에 그 사실을 회원에게 게임 초기 화면이나 게임서비스 홈페이지에 고지합니다.",0.9491,-0.2075
4,287_게임_가공.json,"제15조 (서비스의 제공 및 중단 등) \n③ 회사는 제2항 제1호의 경우, 매주 또는 격주 단위로 일정 시간을 정하여 게임서비스를 중지할 수 있습니다. 이 경우 회사는 최소한 72시간 전에 그 사실을 회원에게 게임 초기 화면이나 게임서비스 홈페이지에 고지합니다.",0.9491,-0.2102


In [24]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)